# Imports

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2
from src.data_utils import unzip, load_data
from src.preprocess import merge, preprocess_ratings, preprocess_movies, preprocess_users

# Prepare data

In [12]:
def get_train_test(base='u1'):
    users, items, ratings_train = load_data(f'{base}.base')
    users, items, ratings_test = load_data(f'{base}.test')

    users, items = preprocess_users(users), preprocess_movies(items)
    ratings_train, ratings_test = preprocess_ratings(ratings_train), preprocess_ratings(ratings_test)

    data_train = merge(ratings_train, users, items)
    data_test = merge(ratings_test, users, items)

    return data_train, data_test

In [ ]:
bases = ['u1','u2','u3','u4','u5', 'ua', 'ub']
data = []

for base in bases:
    data.append(get_train_test(base))

In [10]:
print('Base|Train Movies|Test Movie|Train users|Test users')
for base, (train, test) in zip(bases,data):
    print(base, train['movie_id'].nunique(),test['movie_id'].nunique(), train['user_id'].nunique(),test['user_id'].nunique())

Base|Train Movies|Test Movie|Train users|Test users
u1 1650 1410 943 459
u2 1648 1420 943 653
u3 1650 1423 943 869
u4 1660 1394 943 923
u5 1650 1407 943 927
ua 1680 1129 943 943
ub 1675 1145 943 943


 # Training and testing
 Calculate RMSE and MAE

In [225]:
maes, rmses = [], []
models = {}
for base, (data_train, test_data) in zip(bases, data):
    train, test = data_train.drop(columns=['user_id', 'movie_id']), test_data.drop(columns=['user_id', 'movie_id'])

    X_train = train.drop('rating', axis=1)
    X_test = test.drop('rating', axis=1)
    y_train = train['rating']
    y_test = test['rating']

    # Rescale from 1 to 5
    y_train = y_train*4+1
    y_test = y_test*4+1

    model = RandomForestRegressor(random_state=123)
    model.fit(X_train, y_train)
    models[base]=model
    predictions = model.predict(X_test)

    mae = mean_absolute_error(y_test, predictions)
    rmse = mean_squared_error(y_true=y_test, y_pred=predictions)**0.5

    print(f'Base: {base}, RMSE: {rmse:.4f}. MAE: {mae:.4f}')  

    maes.append(mae)
    rmses.append(rmse)

print(f'Avg MAE: {np.mean(maes)}. Avg RMSE: {np.mean(rmses)}')

Base: u1, RMSE: 1.0205. MAE: 0.8156
Base: u2, RMSE: 1.0060. MAE: 0.8011
Base: u3, RMSE: 0.9963. MAE: 0.7930
Base: u4, RMSE: 1.0041. MAE: 0.7999
Base: u5, RMSE: 1.0066. MAE: 0.8023
Base: ua, RMSE: 1.0311. MAE: 0.8252
Base: ub, RMSE: 1.0429. MAE: 0.8316
Avg MAE: 0.8097977482102579. Avg RMSE: 1.015340711553398


# Check recall

### Functions for produce test data

In [386]:
def combine_with_all_movies(user_id, max_movie=1682, max_time=893286638, exclude_movies=set(), include_movies=None):
    '''Return a dataframe of a user combined with possible movies for prediction of rating'''
    data = []
    for m in range(1, max_movie+1):
        if ((include_movies is not None) and (m not in include_movies)): continue
        elif (m in exclude_movies): continue

        data.append([user_id, m, max_time])

    all_ratings = pd.DataFrame(data, columns=['user_id', 'movie_id', 'timestamp'])

    return all_ratings

def get_for_test(base='u1'):
    '''Returns ready to predict on list of dataframes with the specif user in each combined with possible movies'''
    users, items, ratings_train = load_data(f'{base}.base')
    users, items, ratings = load_data(f'{base}.test')

    users, items = preprocess_users(users), preprocess_movies(items)

    test_users = set(ratings['user_id'])

    data_for_test = []
    for u in test_users:
        # Exclude movies of the user that were in train and include only test
        exclude_movies = set(ratings_train[ratings_train['user_id']==u]['movie_id'])
        include_movies = set(ratings[ratings['user_id']==u]['movie_id'])

        ratings_test = combine_with_all_movies(u, exclude_movies=exclude_movies, include_movies=include_movies)
        ratings_test = preprocess_ratings(ratings_test, has_rating=False)
        
        test_data = merge(ratings_test, users, items)

        data_for_test.append(test_data)

    return data_for_test

## Test recall function
Algo:
- Pick a user
- Iterate over test films and predict a rating
- Pick top K predicted movies based on the rating
- Calculate user recall: Check the present of each predicted film in the list of positively rated movies of this user
- Iterate over all users and average the recall for all user in this test set


In [395]:
def test_recall(base, k=20, liked_rating=4, model=None):
    '''Return average recall for user in the base test set'''
    recalls = []
    for test_user in get_for_test(base=base):
        X_test = test_user

        movie_ids = X_test['movie_id']
        user_id = X_test['user_id'][0]

        X_test = X_test.drop(columns=['user_id', 'movie_id'])

        # Predict rating for each film
        film_rating = [(m, r) for m, r in zip(movie_ids, model.predict(X_test))]
        film_rating = sorted(film_rating, key=lambda t: t[1], reverse=True)

        # Get top k rated films
        top_k = [m for m,_ in film_rating[:k]]

        # Get liked movies by the user
        _, _, ratings = load_data(f'{base}.test')
        liked_movies = list(ratings[(ratings['user_id']==user_id) & \
                                    (ratings['rating']>=liked_rating)]['movie_id'])   

        # Check how many predicted films are in the liked list
        hit = 0
        for m in top_k:
            if m in liked_movies: hit +=1
        
        # print(liked_movies)
        # print(film_rating[:k])
        recall = hit/len(liked_movies) if len(liked_movies) != 0 else 1
        # print(f"Recall for user {user_id}: {recall:.4f}")
        recalls.append(recall)
    return recalls

## Check how many ratings each user gave in the different test sets
Measure mean, median and min to understand what is more less suitable K

In [388]:
for base in bases:
    um = load_data(f'{base}.test')[2][['user_id', "movie_id"]]
    l = list(um.groupby(by=['user_id']))

    sizes = []
    for i in l:
        sizes.append(i[1].shape[0])

    mi, me, min = np.mean(sizes), np.median(sizes), np.min(sizes)
    print(f'Base: {base}, Mean: {mi:f}, Med: {me}, Min: {min}')

Base: u1, Mean: 43.572985, Med: 25.0, Min: 1
Base: u2, Mean: 30.627871, Med: 18.0, Min: 1
Base: u3, Mean: 23.014960, Med: 13.0, Min: 1
Base: u4, Mean: 21.668472, Med: 12.0, Min: 1
Base: u5, Mean: 21.574973, Med: 11.0, Min: 1
Base: ua, Mean: 10.000000, Med: 10.0, Min: 10
Base: ub, Mean: 10.000000, Med: 10.0, Min: 10


# Calculate recall for different test sets

In [393]:
k = 10
liked_rating = 4

total_recall = []
for base in ['u1','u2','u3','u4','u5']:
    recalls = test_recall(base, k=k, liked_rating=liked_rating, model=models[base])
    mean = np.mean(recalls)
    print(f'Base: {base}. Avg recall: {mean:4f}')
    total_recall.append(mean)

print(f'Mean recall on different dataset parts: {np.mean(total_recall):.4f}')
    

Base: u1. Avg recall: 0.535577
Base: u2. Avg recall: 0.636331
Base: u3. Avg recall: 0.715343
Base: u4. Avg recall: 0.742827
Base: u5. Avg recall: 0.757044
Mean recall on different dataset parts: 0.6774
